## 1. 問題分析結果

### 競技プログラミング視点（最速・最小メモリ）

* 1回のカットで増える紙片は **最大でも +1 個**（「同時に複数枚を重ねて切れない」ため）。
* 初期は 1 枚、最終的に必要なピース数は `n*m` 枚。
* よって最小カット回数は **(n*m) - 1** で確定。

### 業務開発視点（型安全・保守性・入力妥当性）

* `n, m` は正の整数前提だが、不正値（0 や負数）が来る可能性まで見るなら例外を投げるのが安全。
* ただし HackerRank では制約内入力が前提なので、実運用ほど厳密にしなくてもOK。

### Python特有考慮（CPython 3.13）

* Python の `int` は任意精度なのでオーバーフローしない。
* 計算は乗算1回＋減算1回の **O(1)**。I/O がボトルネックになり得る問題でもない。

---

## 2. 採用アルゴリズムと根拠

### 結論

* **最小カット数 = n*m - 1**

### 根拠（数学的）

* 各カットは「1つの紙片を2つに分割」する操作なので、紙片数は必ず **+1** 増える。
* 1枚から `n*m` 枚にするには、増分が `n*m - 1` 必要。
* 実現可能性：
  例えば「縦に `n-1` 回切って `n` 本の帯」にし、その後それぞれの帯を横に `m-1` 回ずつ切る…のではなく、**帯を重ねられないので** “全体を一気に切る” 形で

  * 縦方向に `n-1` 回
  * 横方向に `m-1` 回
    …とやるのが誤りに見えますが、実際は「どの順で切っても最終ピース数が `n*m` である限り、必要カット数は必ず `n*m-1`」で固定です（下限＝上限）。

---

## 3. アルゴリズム比較表

| アプローチ              | 時間計算量     | 空間計算量 | Python実装コスト | 可読性 | 標準ライブラリ活用 | CPython最適化 | 備考          |
| ------------------ | --------- | ----- | ----------- | --- | --------- | ---------- | ----------- |
| 方法A: 数式 `n*m-1`    | O(1)      | O(1)  | 低           | ★★★ | 不要        | 適          | 正攻法・最適      |
| 方法B: 切断を逐次シミュレーション | O(n*m) など | O(1)〜 | 中           | ★★☆ | 不要        | 不適         | 不要に遅い、実装も冗長 |

---

## 4. Python特有最適化ポイント

* 乗算＋減算のみ：`return n * m - 1`
* 変数生成やループは不要（最速・最小メモリ）
* 型ヒントで pylance 対応：`def solve(n: int, m: int) -> int`

---

## 5. 実装パターン（2パターン）

* **solve_competitive**：HackerRank想定、最短実装
* **solve_production**：入力妥当性も見る（ただし HackerRank 的には不要）

---

## 6. HackerRankでの回答フォーマット（solve関数を完成）

```python
def solve(n: int, m: int) -> int:
    # Competitive: constraints-valid input assumed
    return n * m - 1
```

---

## 7. 検証（代表ケース）

* サンプル: `n=3, m=1` → `3*1-1=2` ✅
* 最小: `n=1, m=1` → `0`（カット不要）✅
* 大きい値でも Python の int で安全 ✅


# Cutting Paper Squares - 最小カット回数は $nm-1$

## 目次（TOC）
- [概要](#overview)
- [アルゴリズム要点（TL;DR）](#tldr)
- [図解](#figures)
- [証明のスケッチ](#proof)
- [計算量](#complexity)
- [Python 実装](#impl)
- [CPython 最適化ポイント](#cpython)
- [エッジケースと検証](#edgecases)
- [FAQ](#faq)

---

<h2 id="overview">概要</h2>

Mary は $n \times m$ の紙を、$1 \times 1$ の正方形に切り分けたい。

制約は次の通り：

- 1回のカットで切れるのは **紙片1枚だけ**（重ねたり折ったりできない）
- カットは **紙の一辺から反対側の一辺まで**貫く直線
- 目的：合計で $n \cdot m$ 枚の $1 \times 1$ を作るための **最小カット回数**を求める

---

<h2 id="tldr">アルゴリズム要点（TL;DR）</h2>

- 初期状態は紙片が $1$ 枚
- 1回のカットで紙片は必ず **ちょうど 1 枚増える**
  - $1$ 枚を $2$ 枚に分割するので、増分は $+1$
- 最終的に必要な紙片数は $n \cdot m$ 枚
- よって必要な増分は $(n \cdot m) - 1$
- **答え：最小カット回数は $n \cdot m - 1$**

---

<h2 id="figures">図解</h2>

### フローチャート（数学的帰結）

```mermaid
flowchart TD
  Start[開始] --> Input[入力 n と m]
  Input --> Pieces[必要枚数を計算]
  Pieces --> Formula[式 nm-1 を適用]
  Formula --> Output[結果を返す]
````

**説明**: 入力 $n,m$ から必要ピース数 $nm$ を計算し、ピース増加が1回のカットで $+1$ である性質から $nm-1$ を返します。

---

### データフロー（値の流れ）

```mermaid
graph LR
  A[入力 n,m] --> B[計算 nm]
  B --> C[計算 nm-1]
  C --> D[出力]
```

**説明**: データは $n,m \rightarrow nm \rightarrow nm-1$ の順に単純変換されます。

> 注: Mermaid の日本語が表示されない場合は、利用側の CSS で `.mermaid { font-family: "Noto Sans JP", sans-serif; }` のように和文フォントを指定してください。

---

<h2 id="proof">証明のスケッチ</h2>

### 不変条件（紙片数の変化）

* 「1回のカットは紙片1枚を2枚に分割する」操作である
* よって紙片数は常に
  $$
  \text{pieces} \leftarrow \text{pieces} + 1
  $$

### 基底ケース

* 初期状態の紙片数は $1$
  $$
  \text{pieces}_0 = 1
  $$

### 目標状態

* 最終的に必要な紙片数は $n \cdot m$
  $$
  \text{pieces}_\text{goal} = nm
  $$

### 帰納的な増加

* $k$ 回カットしたときの紙片数は
  $$
  \text{pieces}_k = 1 + k
  $$

### 最小回数の導出

* $\text{pieces}_k = nm$ を満たす最小 $k$ を求める：
  $$
  1 + k = nm
  $$
  $$
  k = nm - 1
  $$

### 最適性（下限＝上限）

* 1回のカットで増える紙片は最大でも $1$ 枚なので、$nm-1$ 未満では $nm$ 枚に到達できない（下限）
* 実際に $nm-1$ 回カットすれば必ず $nm$ 枚にできる（上限）
* よって最小値は一意に $nm-1$

---

<h2 id="complexity">計算量</h2>

* 時間計算量：$O(1)$
* 空間計算量：$O(1)$

---

<h2 id="impl">Python 実装</h2>

* HackerRank 形式：`solve(n, m)` を実装
* Pure（副作用なし）：引数から結果を返すだけ
* 型注釈あり（pylance 対応）

```python
from __future__ import annotations

def solve(n: int, m: int) -> int:
    """
    n x m の紙を 1x1 に分割する最小カット回数を返す。

    数学的事実:
      - 1回のカットで増える紙片は +1
      - 初期は 1枚
      - 目標は nm枚
      => k = nm - 1
    """
    # 最小カット回数 = 必要ピース数 - 初期ピース数 = (n*m) - 1
    return n * m - 1
```

---

<h2 id="cpython">CPython 最適化ポイント</h2>

* 乗算 1 回、減算 1 回のみ（ループなし）
* `int` は任意精度のため、制約が大きくてもオーバーフローしない
* 標準ライブラリ不要（関数呼び出しも最小限）

---

<h2 id="edgecases">エッジケースと検証</h2>

| ケース | 入力 $(n,m)$ | 期待値    | 理由                  |
| --- | ---------- | ------ | ------------------- |
| 最小  | $(1,1)$    | $0$    | すでに $1$ 枚なのでカット不要   |
| 一列  | $(n,1)$    | $n-1$  | 必要枚数が $n$ なので $n-1$ |
| 一行  | $(1,m)$    | $m-1$  | 必要枚数が $m$ なので $m-1$ |
| 一般  | $(n,m)$    | $nm-1$ | 1回のカットで $+1$ のため    |

---

<h2 id="faq">FAQ</h2>

### Q1. 縦に $(n-1)$ 回、横に $(m-1)$ 回で合計 $(n+m-2)$ では？

いいえ。この問題では **切れるのは常に紙片1枚だけ**です。

例えば縦に先に切って複数の帯にした後、横に1回切っても「帯をまとめて同時に切る」ことはできません。
そのため、必要な $nm$ 枚を得るには紙片の増分が合計で $nm-1$ 必須になります。

### Q2. $nm-1$ 回で必ず作れるの？

はい。1回カットするたびに紙片が $+1$ されるので、$nm-1$ 回で必ず $nm$ 枚に到達します。

### Q3. Python の `int` で大丈夫？

はい。Python の整数は任意精度なので、$nm$ が非常に大きくても安全に計算できます。

